In [1]:
from elasticsearch import Elasticsearch

In [2]:
# create a object of eleasticsearch
es = Elasticsearch([{'host':'localhost', 'port':9200}])

# Creating index

In [4]:
# how to create elastic search index in python
# amazon
if es.indices.exists(index="amazon_index"):
    print("Index is already present")
else:
    print("Creating index")
    es.indices.create(index="amazon_index")

Creating index


In [5]:
# how to create elastic search index in python
# amazon
if es.indices.exists(index="amazon_index"):
    print("Index is already present")
else:
    print("Creating index")
    es.indices.create(index="amazon_index")

Index is already present


In [6]:
# if you want so see info about a particular index 2 ways
# 1. elastic search head
# 2.http://localhost:9200/name_of_the_index

# Delete a index

In [7]:
es.indices.delete(index="walmart_index")

{'acknowledged': True}

# Different soucrces of data in es

In [8]:
# 1. file --> csv, json
# 2. program
# 3. pandas dataframe

In [11]:
# json kind of structure of your data
data = [
    {
        "title": "Apple iPhone XR (64GB) - Black",
        "review": "Very bad experience with this iPhone xr phone. Back camera found defective and non functional. Amazon is very careless. I haven't got any response on confirmation for replacing. I am just hanging with a damaged piece worth of more than 91000 rupees."
    },
    {
        "title":"Sony 80 cm (32 Inches) HD Ready LED TV KLV-32R202F (Black) (2018 model)",
        "review": "Ultimate picture quality and sound.i just give 4stars due to sony install this 32inch tv on 24inch small tv mount which is not provide any space to plug a pendrive.overall product is good and value for money."
    },
    {
        "title": "Amazon Brand - Solimo Capella Engineered Wood Coffee Table (Espresso Finish)",
        "review": """The table is sturdy and looks good, befitting a modern living room.. The components came in a compact package and the assembly took almost an hour to complete.. Interlocking of the components is very intricate and, therefore, it is recommended to have the product assembled by professionals. The workmanship during assembly was noteworthy. Overall a great product."""
    }
]


In [13]:
# index data in es
from elasticsearch.helpers import bulk
bulk(es,data, index="amazon_index", doc_type="amazon_data")

(3, [])

In [14]:
import pandas as pd
df = pd.DataFrame()
df['title'] = ["Samsung s6 mobile phone", "Nokia 1100 phone"]
df['review'] = ["this phone is good", "nokia is the best phone"]
df

,title,review
0,Samsung s6 mobile phone,this phone is good
1,Nokia 1100 phone,nokia is the best phone


In [18]:
df.to_dict(orient='records')

[{'title': 'Samsung s6 mobile phone', 'review': 'this phone is good'},
 {'title': 'Nokia 1100 phone', 'review': 'nokia is the best phone'}]

In [17]:
df.to_dict()

{'title': {0: 'Samsung s6 mobile phone', 1: 'Nokia 1100 phone'},
 'review': {0: 'this phone is good', 1: 'nokia is the best phone'}}

In [19]:
bulk(es,df.to_dict(orient='records'), index="amazon_index", doc_type="amazon_data")

(2, [])

# Let's search some documents

In [23]:
body_of_request = {
    "query":{
        "match_all":{
            
        }
    }
}

In [28]:
result = es.search(index='amazon_index',body=body_of_request)['hits']['hits']

In [31]:
result = [doc['_source'] for doc in result]

In [32]:
result

[{'title': 'Nokia 1100 phone', 'review': 'nokia is the best phone'},
 {'title': 'Sony 80 cm (32 Inches) HD Ready LED TV KLV-32R202F (Black) (2018 model)',
  'review': 'Ultimate picture quality and sound.i just give 4stars due to sony install this 32inch tv on 24inch small tv mount which is not provide any space to plug a pendrive.overall product is good and value for money.'},
 {'title': 'Apple iPhone XR (64GB) - Black',
  'review': "Very bad experience with this iPhone xr phone. Back camera found defective and non functional. Amazon is very careless. I haven't got any response on confirmation for replacing. I am just hanging with a damaged piece worth of more than 91000 rupees."},
 {'title': 'Amazon Brand - Solimo Capella Engineered Wood Coffee Table (Espresso Finish)',
  'review': 'The table is sturdy and looks good, befitting a modern living room.. The components came in a compact package and the assembly took almost an hour to complete.. Interlocking of the components is very intri

# Query wise retrieval

In [48]:
query = "xr camera"

In [49]:
body_of_request = {
    "query":{
        "match":{
            "review":query
        }
    }
}

In [50]:
query_res = es.search(index="amazon_index", body=body_of_request)['hits']['hits']

In [51]:
[doc['_source'] for doc in query_res]

[{'title': 'Apple iPhone XR (64GB) - Black',
  'review': "Very bad experience with this iPhone xr phone. Back camera found defective and non functional. Amazon is very careless. I haven't got any response on confirmation for replacing. I am just hanging with a damaged piece worth of more than 91000 rupees."}]

In [52]:
# Randing model
# query = ""
# reviews = []

In [53]:
# OR
# "apple" OR "samsung"

In [54]:
body_of_request = {
    "query":{
        "bool":{
            "must":{
                "multi_match":{
                    "query":"(apple) OR (samsung)",
                    "operator":"or",
                    "fields":[
                        "title",
                        "review"
                    ]
                }
            }
        }
    }
}

In [56]:
es.search(index="amazon_index", body=body_of_request)['hits']['hits']

[{'_index': 'amazon_index',
  '_type': 'amazon_data',
  '_id': 'Sd3-TW4BW95tfB-cODy8',
  '_score': 1.1548918,
  '_source': {'title': 'Samsung s6 mobile phone',
   'review': 'this phone is good'}},
 {'_index': 'amazon_index',
  '_type': 'amazon_data',
  '_id': 'Rt32TW4BW95tfB-c1TzD',
  '_score': 1.0732633,
  '_source': {'title': 'Apple iPhone XR (64GB) - Black',
   'review': "Very bad experience with this iPhone xr phone. Back camera found defective and non functional. Amazon is very careless. I haven't got any response on confirmation for replacing. I am just hanging with a damaged piece worth of more than 91000 rupees."}}]